In [ ]:
from torch import cuda, bfloat16
import transformers

# model_id = 'meta-llama/Llama-2-7b-chat-hf'
model_id = 'openlm-research/open_llama_3b_v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device

In [ ]:
# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
# begin initializing HF items, need auth token for these
hf_auth = ''
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights

with init_empty_weights():
    model = transformers.AutoModelForCausalLM.from_config(model_config)

device_map = infer_auto_device_map(model, no_split_module_classes=["LlamaDecoderLayer"])
device_map

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")